In [1]:
import glob
from codecs import open

import pandas as pd
import numpy as np
from google.cloud import language
from google.cloud.language import enums
from google.cloud.language import types

%env GOOGLE_APPLICATION_CREDENTIALS=/Users/emersonsjsu/GitHub/EarlGrey/google_private_key.json
WORKING_DIRECTORY = "/Users/emersonsjsu/GitHub/EarlGrey"

env: GOOGLE_APPLICATION_CREDENTIALS=/Users/emersonsjsu/GitHub/EarlGrey/google_private_key.json


In [22]:
# Clear char_df
char_df = char_df.iloc[0:0]
FIRST_EPISODE_OFFSET = 101


def sanitize_name(name_line):
    name = name_line.strip()  # Voice overs count as same character
    if name.endswith('V.O.'):
        name = name[:-5]
    # Strip parentheticals from name
    if name.find('(') != -1:
        name = name[:name.find('(')]
    # Strip 'S VOICE from name
    if name.endswith('\'S VOICE'):
        name = name[:-8]
    # Strip 'S COM VOICE from name
    if name.endswith('\'S COM VOICE'):
        name = name[:-12]
    return name.strip()


def process_script(file_path):
    current_file = open(file_path, "r", encoding='utf-8', errors="ignore")
    episode_no = int(current_file.name[current_file.name.rfind('/') + 1:-4]) - FIRST_EPISODE_OFFSET
    is_dialogue = False
    text_array = []
    character_array = []
    lines = current_file.readlines()
    for line in lines:
        # Character dialogue has ended, reset current character
        if line.strip() == '':
            is_dialogue = False
        # All names are preceded by exactly 5 tabs
        elif line[:5] == '\t\t\t\t\t' and line[5:6] != '\t' and not line.strip().startswith('('):
            # We have found a name! Update current name
            text_array.append([])
            character_array.append(sanitize_name(line))
            character_name = sanitize_name(line)
        # If we are still looking at a character's dialogue
        elif is_dialogue:
            # Append dialogue to character's current dialogue list
            text_array[-1].append(' %s' % line.strip())
    text_array = map(lambda x: ''.join(x)[1:], text_array)
    return text_array, character_array


text_array, character_array = process_script(WORKING_DIRECTORY + '/scripts/225.txt')
print(len(text_array))


384


In [25]:
def google_annotate(text):
    client = language.LanguageServiceClient()
    document = types.Document(
        language='EN',
        content=text,
        type=enums.Document.Type.PLAIN_TEXT
    )
    annotations = client.analyze_sentiment(document=document)
    return annotations


annotation_array = map(google_annotate, text_array)
annotation_array[0]


document_sentiment {
}
language: "en"

In [29]:
#my_index = pd.MultiIndex(levels=[range(len(annotation_array)), character_array],
                         #labels=[[], []],
                         #names=[u'dialogue_number', u'character'])
#my_columns = [u'text',u'sentiment_score',u'sentiment_magnitude']
#char_df = pd.DataFrame(index=my_index, columns=my_columns)
arrays = range(len(annotation_array)), character_array
my_index = pd.MultiIndex.from_arrays(arrays=arrays, names=('dialogue_index','character'))
pd.DataFrame(index=my_index)


,
dialogue_index,character
0,PICARD
1,PICARD
2,GEORDI
3,PICARD
4,RIKER
5,PICARD
6,WORF
7,PICARD
8,PICARD


In [181]:
char_df.to_pickle(WORKING_DIRECTORY+'/char_df.pickle')

In [207]:
temp = pd.read_pickle(WORKING_DIRECTORY+'/char_df.pickle')
temp[temp['sentiment_score'] > 0].loc['PICARD'].sort_values(by=['sentiment_score','sentiment_magnitude','episode_no'],
                                                            ascending=[False, False, True])


,text,sentiment_score,sentiment_magnitude
episode_no,,,
39,"Captain's log, Stardate 42686.4. We are en rou...",0.2,15.600000
172,"As a result, the Kearsarge won't be able to re...",0.2,7.100000
132,"Captain's Log, Stardate 46235.7 Ensign Ro, Kei...",0.2,5.000000
167,What's the last thing you remember?,0.2,0.200000
3,"Captain's log, stardate 41235.25. Our location...",0.1,48.700001
144,"Good evening, Mister Data. No need to report. ...",0.1,48.599998
44,"Captain's log, Stardate 42881.5. We are in orb...",0.1,48.099998
82,"Captain's log, stardate 44307.3. I am preparin...",0.1,45.200001
31,"Captain's log, Stardate 42411.2. The Enterpris...",0.1,44.299999


In [28]:

character_array

[u'PICARD',
 u'PICARD',
 u'GEORDI',
 u'PICARD',
 u'RIKER',
 u'PICARD',
 u'WORF',
 u'PICARD',
 u'PICARD',
 u'PICARD',
 u'DATA',
 u'RIKER',
 u'WORF',
 u'DATA',
 u'GEORDI',
 u'WORF',
 u'RIKER',
 u'DATA',
 u'WORF',
 u'PICARD',
 u'RIKER',
 u'RIKER',
 u'ELINE',
 u'ELINE',
 u'ELINE',
 u'PICARD',
 u'ELINE',
 u'PICARD',
 u'PICARD',
 u'ELINE',
 u'PICARD',
 u'ELINE',
 u'PICARD',
 u'ELINE',
 u'PICARD',
 u'ELINE',
 u'PICARD',
 u'ELINE',
 u'ELINE',
 u'ELINE',
 u'BATAI',
 u'BATAI',
 u'PICARD',
 u'BATAI',
 u'PICARD',
 u'BATAI',
 u'PICARD',
 u'BATAI',
 u'PICARD',
 u'BATAI',
 u'PICARD',
 u'BATAI',
 u'PICARD',
 u'BATAI',
 u'PICARD',
 u'BATAI',
 u'PICARD',
 u'BATAI',
 u'PICARD',
 u'BATAI',
 u'PICARD',
 u'BATAI',
 u'PICARD',
 u'BATAI',
 u'PICARD',
 u'PICARD',
 u'BATAI',
 u'PICARD',
 u'ELINE',
 u'ELINE',
 u'ELINE',
 u'PICARD',
 u'ELINE',
 u'PICARD',
 u'ELINE',
 u'ELINE',
 u'PICARD',
 u'ELINE',
 u'PICARD',
 u'ELINE',
 u'PICARD',
 u'ELINE',
 u'PICARD',
 u'PICARD',
 u'ELINE',
 u'PICARD',
 u'ELINE',
 u'PICARD',

In [ ]:
temp.head()